In [9]:
import pandas as pd
import re

In [10]:
df = pd.read_excel('EquityIndexOptionsHistTrading-2021-2024.xlsx')

In [11]:
df

,Series,Trade Date,Open,High,Low,Close,Prior Settlement,Settlement,Change,%Change
0,S50F21C1000,2021-01-04,-,-,-,-,4.6,4.9,0.3,6.52
1,S50F21C1025,2021-01-04,-,-,-,-,2.4,2.5,0.1,4.17
2,S50F21C1050,2021-01-04,-,-,-,-,1.2,1.1,-0.1,-8.33
3,S50F21C650,2021-01-04,-,-,-,-,258.4,272.6,14.2,5.50
4,S50F21C675,2021-01-04,-,-,-,-,233.4,247.6,14.2,6.08
...,...,...,...,...,...,...,...,...,...,...
90327,S50M25P875,2024-12-30,18.5,19.5,18.4,19.5,21.5,19.5,-2.0,-9.30
90328,S50M25P900,2024-12-30,30,30,30,30,30.7,30.0,-0.7,-2.28
90329,S50M25P925,2024-12-30,-,-,-,-,44.5,45.1,0.6,1.35
90330,S50M25P950,2024-12-30,-,-,-,-,61.0,62.1,1.1,1.80


In [12]:
def get_otm_options(df, series_col='Series'):
    """
    Retrieves the 4 most out-of-the-money (OTM) options for each option series,
    distinguishing between calls and puts.

    Args:
        df (pd.DataFrame): Input DataFrame containing option data.
        series_col (str):  Name of the column containing option series.
                           Assumes the first 7 characters are the series, and
                           the 7th character indicates option type (C/P).

    Returns:
        pd.DataFrame: DataFrame containing the 4 most OTM options for each series.
    """

    def extract_strike_price(series_name):
        """Extracts the numeric strike price from the series name."""
        match = re.search(r'^.{7}(\D*)(\d+)$', series_name)
        if match:
            return int(match.group(2))
        else:
            return float('inf')

    df['StrikePrice'] = df[series_col].apply(extract_strike_price)
    df['Series7'] = df[series_col].str.slice(0, 7)
    df['OptionType'] = df[series_col].str[6].str.upper()  # Ensure 'C' or 'P'

    def get_otm_n_for_series(series_group):
        """Gets the n most OTM options for a series group."""
        if series_group['OptionType'].iloc[0] == 'C':
            return series_group.sort_values(by='StrikePrice', ascending=True).head(4)  # Lowest for Calls
        elif series_group['OptionType'].iloc[0] == 'P':
            return series_group.sort_values(by='StrikePrice', ascending=False).head(4) # Highest for Puts
        else:
            return pd.DataFrame()  # Return empty DataFrame for unknown type

    result_df = df.groupby(['Series7', 'OptionType']).apply(get_otm_n_for_series)

    return result_df.reset_index(level=[0, 1], drop=True)


# --- Example Usage ---
if __name__ == "__main__":
    # Load the data from the Excel file
    option_data = pd.read_excel('EquityIndexOptionsHistTrading-2021-2024.xlsx')

    result_df = get_otm_options(option_data)
    result_df = result_df[['Series', 'StrikePrice', 'OptionType']]  # Keep only relevant columns
    result_df.reset_index(drop=True, inplace=True)  # Reset index
    print(result_df.head(10))

        Series  StrikePrice OptionType
0   S50F21C650          650          C
1   S50F21C650          650          C
2   S50F21C650          650          C
3   S50F21C650          650          C
4  S50F21P1075         1075          P
5  S50F21P1075         1075          P
6  S50F21P1075         1075          P
7  S50F21P1075         1075          P
8   S50F22C825          825          C
9   S50F22C825          825          C


/var/folders/rc/w7r40l35143bxwkzfxdygctc0000gn/T/ipykernel_10771/3460359731.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = df.groupby(['Series7', 'OptionType']).apply(get_otm_n_for_series)


In [13]:
hist = pd.read_csv('SET 50 Historical Data (5).csv')
hist['Date'] = pd.to_datetime(hist['Date'], format='%m/%d/%Y')
hist = hist[['Date', 'Price']]
hist.head(5)

,Date,Price
0,2024-12-30,906.23
1,2024-12-27,908.90
2,2024-12-26,906.41
3,2024-12-25,907.17
4,2024-12-24,904.44


In [14]:
endDate = pd.read_excel('SOMC Data SET Clear.xlsx', sheet_name='Option Series')
endDate['END_DATE'] = pd.to_datetime(endDate['END_DATE'], format='%Y%m%d')
endDate = endDate[['NAME', 'END_DATE']]
endDate.head(5)

,NAME,END_DATE
0,S50K14C1000,2014-05-30
1,S50K14C825,2014-05-30
2,S50K14C850,2014-05-30
3,S50K14C875,2014-05-30
4,S50K14C900,2014-05-30


In [15]:
mer = result_df.merge(endDate, how='inner', left_on='Series', right_on='NAME', suffixes=('', '_y'))
mer = mer[['Series', 'OptionType', 'StrikePrice', 'END_DATE']]
finalDF = mer.merge(hist, how='inner', left_on='END_DATE', right_on='Date', suffixes=('', '_y'))
finalDF = finalDF[['Series', 'OptionType', 'END_DATE', 'StrikePrice', 'Price']]
finalDF.head(10)

,Series,OptionType,END_DATE,StrikePrice,Price
0,S50F21C650,C,2021-01-29,650,916.66
1,S50F21C650,C,2021-01-29,650,916.66
2,S50F21C650,C,2021-01-29,650,916.66
3,S50F21C650,C,2021-01-29,650,916.66
4,S50F21P1075,P,2021-01-29,1075,916.66
5,S50F21P1075,P,2021-01-29,1075,916.66
6,S50F21P1075,P,2021-01-29,1075,916.66
7,S50F21P1075,P,2021-01-29,1075,916.66
8,S50F22C825,C,2022-01-31,825,993.29
9,S50F22C825,C,2022-01-31,825,993.29


In [16]:
finalDF['StrikePrice'] = finalDF['StrikePrice'].astype(float)
finalDF['Price'] = finalDF['Price'].str.replace(',', '', regex=False).astype(float)
finalDF['P/L'] = finalDF.apply(lambda row: max(row['Price'] - row['StrikePrice'], 0) 
                          if row['OptionType'] == 'C' 
                          else max(row['StrikePrice'] - row['Price'], 0), axis=1)
finalDF['P/L'] = finalDF['P/L'].round(2)
finalDF

,Series,OptionType,END_DATE,StrikePrice,Price,P/L
0,S50F21C650,C,2021-01-29,650.0,916.66,266.66
1,S50F21C650,C,2021-01-29,650.0,916.66,266.66
2,S50F21C650,C,2021-01-29,650.0,916.66,266.66
3,S50F21C650,C,2021-01-29,650.0,916.66,266.66
4,S50F21P1075,P,2021-01-29,1075.0,916.66,158.34
...,...,...,...,...,...,...
407,S50Z24C700,C,2024-12-30,700.0,906.23,206.23
408,S50Z24P1050,P,2024-12-30,1050.0,906.23,143.77
409,S50Z24P1050,P,2024-12-30,1050.0,906.23,143.77
410,S50Z24P1050,P,2024-12-30,1050.0,906.23,143.77
